### Training a Regularized Logistic Regression model for Machine Failure Prediction using NumPy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
import pickle as pkl

### Exploring the Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/machine-failure-prediction-using-sensor-data/data.csv') # importing the data
load = False

# Optional: load the trained parameters
'''
with open('trained_parameters.pkl', 'rb') as file:
    loaded_parameters = pkl.load(file)

    # Extract parameters and hyperparameters
    w = loaded_parameters['weights']
    b = loaded_parameters['bias']
    epochs =  loaded_parameters['epochs']
    alpha = loaded_parameters['learning rate']
    lambda_reg = loaded_parameters['reg parameter']
    
    load = True
'''

In [ ]:
df.describe() # getting the summary of the data

In [ ]:
df.info() # getting the information of the data regarding the data types of features, number of non-null values, etc

In [ ]:
df.duplicated().sum() # checking for the number of duplicate rows in the data

In [ ]:
df.head() # getting the first 5 rows of the data

My initial impressions of the dataset:

* There are 944 total rows, with 1 duplicated row.
* All data is in numeric form, so there isn't much data pre-processing needed.
* There are no missing values.

In [ ]:
df.drop_duplicates(inplace=True) # dropping the duplicate rows

### Checking Skewness of Dataset

Checking if the dataset is imbalanced and skewed in any way

In [ ]:
color = ['#92B6B1', '#43C59E'] # defining the colors for the plots
sns.set_style('darkgrid') # setting the style for the plot
df['fail'].value_counts().plot.bar(color=color) # plotting the bar plot for the target variable
plt.xlabel('Fail') # setting the x-axis label
plt.ylabel('Total') # setting the y-axis label
plt.show()

It looks like the data is skewed, as the count of positive examples are lower than the negative examples. I will use SMOTE, an over sampling technique to increase the count of positive examples to make the distribution a little better.

### Over sampling to decrease Skewness

In [ ]:
over_sampler = SMOTE(random_state=42) # initializing the SMOTE object
X = df.drop(['fail'], axis=1) # seperating the features from the target feature
y = df['fail'] # isolating the target feature
X, y = over_sampler.fit_resample(X, y) # resampling the dataset

In [ ]:
y.value_counts().plot.bar(color=color) # plotting the bar plot for the target variable after resampling

As you can see, the count of positive and negative examples is much better. Now I will scale the dataset and then train a model.

### Feature Scaling

I will use a min-max scaler, it will scale the values of features to a value between the range 0 and 1.

In [ ]:
scaler = MinMaxScaler() # Initializing the MinMaxScaler
X_scaled = scaler.fit_transform(X) # scaling the data

### Data Splitting

Next, we will split the data into Test and Train sets. This is an important step. You want a way to evaluate the model, and splitting the data into Train and Test sets gives you that option. Usually, you also would want to split the Test set further into Cross-validation and Test sets, and use the Cross-validation set for hyperparameter tuning. I used that approach to tune the hyperparameters, and now that I have finalized the values for hyperparameters, I will skip splitting the data further into cross-validation set and test set, and just directly split the data into Training and Test sets. I will do a 70:30 split on the dataset.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42) # splitting the data into training and testing sets

### What is Regularization?

To understand regularization, fist let's understand what is overfitting. Overfitting is an undesirable behavior in Machine Learning in which a ML algorithm doesn't generalize well to all types of data that can occur, and fits too closely to the training data. In other words, it gives too much emphasis on the training data, so much so that if it encounters any other type of data that is a little different to the data it "learned", it is unable to give a satisfactory prediction. This phenomena can also be stated as the model having "High Variance".

This is where regularization comes in. Regularization prevents ovefitting by adding a penalty term in the cost function, which discourages the model to give to much importance to the training data. There are three types of regularization, but I have used the L2 Regularization, also called Ridge regularization. In this type of regularization, we add the sqaured sum of the magnitudes of the weights in the cost function. Regularization also adds a hyperparameter called lambda, which is a regularization parameter which basically affects the effect the penalty term makes on the overall cost function.

### Model Training

I will be training a regularized logistic regression model. I will implement everything from scratch, using just numpy and maths. I will be using the Binary Cross Entropy cost function, and I will be optimizing it using Gradient Descent. The functions for gradient descent and others are defined below.

In [ ]:
def compute_cost(w, b, X, y, lambda_reg):
    '''
    Compute the cost function for logistic regression
    
    Parameters:
        w (numpy array): weight parameters of the model
        b (float): bias parameter of the model
        X (numpy array): features of the dataset excluding the target feature
        y (numpy array): target feature of the dataset
        lambda_reg (int): regularization parameter
    
    Returns:
        cost (float): the cost of the model with respect to the current weights and bias
    '''
    m = X.shape[0] # getting the number of examples in the dataset
    z = np.dot(X, w) + b # computing the linear combination of the weights and features
    f_wb = 1/(np.exp(-z) + 1) # applying the sigmoid function to the linear combination
    cost = (-1/m) * np.sum(y * np.log(f_wb) + (1-y) * np.log(1-f_wb)) + (lambda_reg/(2*m) * np.sum(w**2)) # computing the cost function
    return cost


def gradient_descent(w, b, X, y, lambda_reg):
    '''
    Compute the derivatives of the cost function with respect to the weights and bias.
    
    Parameters:
        w (numpy array): weight parameters of the model
        b (float): bias parameter of the model
        X (numpy array): features of the dataset excluding the target feature
        y (numpy array): target feature of the dataset
        lambda_reg (int): regularization parameter
    
    Returns:
        dj_dw (numpy array): the value of the derivative of the cost function with respect to the weights
        dj_db (float): the value of the derivative of the cost function with respect to the bias
    '''
    m, n = X.shape # getting the number of examples and features in the dataset
    z = np.dot(X, w) + b # computing the linear combination of the weights and features
    f_wb = 1/(np.exp(-z) + 1) # applying the sigmoid function to the linear combination
    err = f_wb - y  # computing the error term
    dj_dw = (1/m) * np.dot(X.T, err) + (lambda_reg/m) * w # computing the derivative of the cost function with respect to the weights, and adding the regularization term
    dj_db = (1/m) * np.sum(err) # computing the derivative of the cost function with respect to the bias. Since I am only regularizing the weights, I will skip adding the regularization term to the bias. Although, that is an option that you can implement
    return dj_dw, dj_db


def compute_GD(w, b, X, y, epochs, alpha, lambda_reg, stopping_criteria=0.00001):
    '''
    Implemented gradient descent algorithm to find the optimal weights and bias for the logistic regression model.

    Parameters:
        w (numpy array): weight parameters of the model
        b (float): bias parameter of the model
        X (numpy array): features of the dataset excluding the target feature
        y (numpy array): target feature of the dataset
        epochs (int): number of iterations to run the gradient descent algorithm
        alpha (float): learning rate
        lambda_reg (int): regularization parameter
        stopping_criteria (float): the difference between the cost of the current iteration and the previous iteration to stop the algorithm

    Returns:
        w (numpy array): the optimal weight parameters of the model
        b (float): the optimal bias parameter of the model
        JD_history (list): the cost of the model at each iteration for visualization purposes
    '''
    JD_history = [] # initializing the list to store the cost of the model at each iteration, for visualization purposes
    
    for i in range(epochs): # looping through the number of iterations
        dj_dw, dj_db = gradient_descent(w, b, X, y, lambda_reg) # computing the derivatives of the cost function with respect to the weights and bias
        w = w - alpha * dj_dw # updating the weights
        b = b - alpha * dj_db # updating the bias
        
        if i<100000: # storing the cost of the model at each iteration
            JD_history.append(compute_cost(w, b, X, y, lambda_reg)) # computing the cost of the model at each iteration
        
        if i% math.ceil(epochs/10) == 0: # printing the cost of the model at every 100th iterations
            print("Epoch:{:4}, Loss:{:4.2f}".format(i, JD_history[-1])) # printing the cost and iteration number
        
        if i>1 and abs(JD_history[-1] - JD_history[-2]) < stopping_criteria: # checking if the difference between the cost of the current iteration and the previous iteration is less than the stopping criteria
            break
    
    return w, b, JD_history

In [ ]:
if not load:
    # Defining the hyperparameters

    epochs = 10000 # number of iterations
    alpha = 0.01 # learning rate
    lambda_reg = 1 # regularization parameter

    w = np.zeros(X_train.shape[1]) # initializing the weights
    b = 0 # initializing the bias

    w, b, JD_history = compute_GD(w, b, X_train, y_train, epochs, alpha, lambda_reg) # training the model

In [ ]:
plt.plot(JD_history) # plotting the cost of the model at each iteration
plt.xlabel('Iterations') # setting the x-axis label
plt.ylabel('Loss') # setting the y-axis label

The loss seems to be decreasing at a decent pace, there are no signs that we are shooting over the global optimum (the loss is constantly decreasing), which would have meant our learning rate is too high.

### Evaluating Model

I will be evaluating the model using classical classification metrics like accuracy, precision, recall, etc. I will also generate a classification report and a confusion matrix so that we know the performance of the model for each class as well. I am setting the threshold of the model to 0.5. We are using a threshold because when we apply the sigmoid function to the output to the linear combination of weights and bias, it generates a value between 0 and 1, which is basically a probability of the example being of class 1. By applying a threshold, we are setting the criteria for the probability that an example should have to be labelled as a positive example (or belonging to class 1). If the probability of an example is 0.5 or above, it is labelled as 1. Anything otherwise is labelled 0.

In [ ]:
y_test_pred = [] # initializing the list to store the predictions of the model
threshold = 0.5 # setting the threshold for the model
m = X_test.shape[0] # getting the number of examples in the test set
for i in range(m):
    z = np.dot(X_test[i], w) + b # computing the linear combination of the weights and features
    f_wb = 1/(np.exp(-z) + 1) # applying the sigmoid function to the linear combination
    y_test_pred.append(1 if f_wb >= threshold else 0) # making the prediction based on the threshold

#### Confusion Matrix

In [ ]:
conf_matrix = confusion_matrix(y_test, y_test_pred) # computing the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix) # initializing the confusion matrix display
disp.plot() # plotting the confusion matrix

#### Classification Report

In [ ]:
class_report = classification_report(y_test, y_test_pred) # computing the classification report
print("Classification Report: \n", class_report) # printing the classification report

In [ ]:
y_test_pred = np.array(y_test_pred) # converting the predictions to a numpy array
accuracy = accuracy_score(y_test, y_test_pred) # computing the accuracy of the model
precision = precision_score(y_test, y_test_pred) # computing the precision of the model
recall = recall_score(y_test, y_test_pred) # computing the recall of the model
f1 = f1_score(y_test, y_test_pred) # computing the f1 score of the model

In [ ]:
scores = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1} # storing the scores in a dictionary

color = ['#92B6B1', '#43C59E', '#2F4858', '#33658A'] # defining the colors for the plot
sns.set_style('darkgrid') # setting the style for the plot
plt.figure(figsize=(10, 6)) # setting the figure size
plt.bar(scores.keys(), scores.values(), color=color) # plotting the bar plot for the scores
plt.ylabel('Score') # setting the y-axis label
plt.ylim(0, 1)  # setting the range of values to be shown on the y-axis, which is between 0 and 1
plt.title('Evaluation Scores') # setting the title of the plot
plt.show() # displaying the plot

In [ ]:
# Save the trained parameters
if not load:
    parameters = {
        'weights': w,
        'bias': b,
        'epochs': epochs,
        'learning rate': alpha,
        'reg parameter': lambda_reg
    }

    with open('trained_parameters.pkl', 'wb') as file:
        pkl.dump(parameters, file)